In [1]:
!pip install pennylane
!pip isntall torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 14.3 MB/s eta 0:00:00
ERROR: unknown command "isntall" - maybe you meant "install"


In [2]:
import pennylane as qml  # Importing PennyLane for quantum computing
from pennylane import numpy as np  # Importing PennyLane's NumPy for compatibility
import torch  # Importing PyTorch for potential machine learning applications
import torch.nn as nn  # Importing PyTorch's neural network module
import copy  # Importing copy module for deep copying objects
import itertools  # Importing itertools for combinatorial operations
from tqdm import tqdm  # Importing tqdm for progress tracking

class AdamOptimizer():
	"""Stochastic gradient descent optimizer with Adam
	Note: All default values are from the original Adam paper
	Parameters
	----------
	params : list, length = len(coefs_) + len(intercepts_)
		The concatenated list containing coefs_ and intercepts_ in MLP model.
		Used for initializing velocities and updating params
	learning_rate_init : float, default=0.001
		The initial learning rate used. It controls the step-size in updating
		the weights
	beta_1 : float, default=0.9
		Exponential decay rate for estimates of first moment vector, should be
		in [0, 1)
	beta_2 : float, default=0.999
		Exponential decay rate for estimates of second moment vector, should be
		in [0, 1)
	epsilon : float, default=1e-8
		Value for numerical stability
	Attributes
	----------
	learning_rate : float
		The current learning rate
	t : int
		Timestep
	ms : list, length = len(params)
		First moment vectors
	vs : list, length = len(params)
		Second moment vectors
	References
	----------
	Kingma, Diederik, and Jimmy Ba.
	"Adam: A method for stochastic optimization."
	arXiv preprint arXiv:1412.6980 (2014).
	"""

	def __init__(self, params, learning_rate_init=0.001, beta_1=0.9,
				 beta_2=0.999, epsilon=1e-8, amsgrad = False):

		self.beta_1 = beta_1
		self.beta_2 = beta_2
		self.epsilon = epsilon
		if type(learning_rate_init) == float:
			self.learning_rate_init = np.ones(len(params))*learning_rate_init
		else:
			self.learning_rate_init = np.array(learning_rate_init)
		self.t = 0
		self.ms = [np.zeros_like(param) for param in params]
		self.vs = [np.zeros_like(param) for param in params]
		self.amsgrad = amsgrad
		self.max_vs = [np.zeros_like(param) for param in params]

	def get_updates(self, grads):
		"""Get the values used to update params with given gradients
		Parameters
		----------
		grads : list, length = len(coefs_) + len(intercepts_)
			Containing gradients with respect to coefs_ and intercepts_ in MLP
			model. So length should be aligned with params
		Returns
		-------
		updates : list, length = len(grads)
			The values to add to params
		"""
		self.t += 1
		self.ms = [self.beta_1 * m + (1 - self.beta_1) * grad
					 for m, grad in zip(self.ms, grads)]
		self.vs = [self.beta_2 * v + (1 - self.beta_2) * (grad ** 2)
					 for v, grad in zip(self.vs, grads)]
		self.max_vs = [np.maximum(v, max_v) for v, max_v in zip(self.vs, self.max_vs)]
		self.learning_rate = (self.learning_rate_init *
								np.sqrt(1 - self.beta_2 ** self.t) /
								(1 - self.beta_1 ** self.t))
		if self.amsgrad:
			updates = [lr * m / (np.sqrt(max_v) + self.epsilon)
			           for lr, m, max_v in zip(self.learning_rate, self.ms, self.max_vs)]
		else:
			updates = [lr * m / (np.sqrt(v) + self.epsilon)
					 for lr, m, v in zip(self.learning_rate, self.ms, self.vs)]
		return updates



class QAOA_layer():
    def __init__(self, depth, Q):
        """
        Initializes the QAOA layer with a given depth and QUBO matrix.

        Args:
            depth (int): The depth of the QAOA circuit (number of layers).
            Q (np.ndarray): QUBO matrix representing the quadratic unconstrained binary optimization problem.
        """
        self.Q = Q  # Store the QUBO matrix
        self.p = depth  # Store the QAOA depth
        self.ham = self.prepare_cost_hamiltonian()  # Prepare the cost Hamiltonian based on QUBO matrix
        self.dev = qml.device("default.qubit", wires=Q.shape[0])  # Define a quantum device with qubits equal to the size of Q

    def qaoa_circuit(self, params):
        """
        Constructs the QAOA circuit with the given parameters.

        Args:
            params (list): A list containing gamma and beta values for parameterized QAOA layers.
        """
        n = self.Q.shape[0]  # Number of qubits based on QUBO matrix size
        gammas = params[:self.p]  # Extract gamma parameters for cost Hamiltonian evolution
        betas = params[self.p:]  # Extract beta parameters for mixer Hamiltonian
        # Apply Hadamard gates to all qubits to initialize in uniform superposition
        for i in range(n):
            qml.Hadamard(wires=i)

        # Apply QAOA layers consisting of cost and mixer Hamiltonians
        for layer in range(self.p):
            self.qubo_cost(gammas[layer])  # Apply cost Hamiltonian with corresponding gamma
            self.mixer(betas[layer])  # Apply mixer Hamiltonian with corresponding beta

    def qubo_cost(self, gamma):
        """
        Implements the cost Hamiltonian evolution for the QUBO problem.

        Args:
            gamma (float): Parameter for cost Hamiltonian evolution.
        """
        n = self.Q.shape[0]  # Get number of qubits
        for i in range(n):
            for j in range(n):
                if self.Q[i, j] != 0:  # If the QUBO coefficient is non-zero
                    if i == j:
                        qml.RZ(2 * gamma * float(self.Q[i, j]), wires=i)  # Apply single-qubit phase rotation
                    else:
                        qml.CNOT(wires=[i, j])  # Apply CNOT before controlled rotation
                        qml.RZ(2 * gamma * float(self.Q[i, j]), wires=j)  # Apply controlled phase rotation
                        qml.CNOT(wires=[i, j])  # Undo entanglement with another CNOT

    def mixer(self, beta):
        """
        Implements the mixer Hamiltonian for QAOA.

        Args:
            beta (float): Parameter for mixer Hamiltonian evolution.
        """
        for i in range(self.Q.shape[0]):
            qml.RX(2 * beta, wires=i)  # Apply X-rotation to all qubits

    def prepare_cost_hamiltonian(self):
        """
        Constructs the cost Hamiltonian for the QUBO problem.

        Returns:
            qml.Hamiltonian: The constructed cost Hamiltonian.
        """
        n = self.Q.shape[0]  # Get the size of the QUBO matrix
        coeffs = []  # Store the coefficients of the Hamiltonian terms
        ops = []  # Store the corresponding Pauli operators

        for i in range(n):
            for j in range(n):
                if self.Q[i, j] != 0:  # Only consider non-zero QUBO elements
                    if i == j:
                        coeffs.append(self.Q[i, j])  # Add coefficient for single-qubit term
                        ops.append(qml.PauliZ(i))  # Add Pauli-Z operator
                    else:
                        coeffs.append(self.Q[i, j])  # Add coefficient for two-qubit interaction
                        ops.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Add ZZ interaction term

        return qml.Hamiltonian(coeffs, ops)  # Return the constructed Hamiltonian



class RL_QAOA():
    def __init__(self, Q, n_c,init_paramter,b_vector, QAOA_depth,gamma=0.99,learning_rate_init=0.001):
        self.Q = Q
        self.n_c = n_c
        self.param = init_paramter
        self.b = b_vector
        self.p = QAOA_depth
        self.qaoa_layer = QAOA_layer(QAOA_depth,Q)
        self.gamma = gamma
        self.optimzer = AdamOptimizer([init_paramter,b_vector],learning_rate_init=learning_rate_init)




    def RL_QAOA(self,episodes,epochs,correc_ans = None):
        ## RL_QAOA실행 episodes : 몬테카를로를 진행할 숫자
        ## epochs : parameter 최적화를 진행할 숫자

        for j in range(epochs):
          value_list = []
          QAOA_diff_list = []
          beta_diff_list = []
          if correc_ans is not None:
            prob = 0


          for i in tqdm(range(episodes)):
            QAOA_diff,beta_diff,value = self.rqaoa_execute()
            value_list.append(value)
            QAOA_diff_list.append(QAOA_diff)
            beta_diff_list.append(beta_diff)

            if correc_ans is not None:
              if value <= correc_ans+0.01 and value >= correc_ans-0.01:
                prob+=1


          batch_maen = np.array(value_list) - np.mean(np.array(value_list),axis=0)
          print(batch_maen)

          for index,val in enumerate(batch_maen):
            QAOA_diff_list[index] *= -val
            beta_diff_list[index] *=-val


          QAOA_diff_sum = np.mean(QAOA_diff_list,axis=0)
          beta_diff_sum = np.mean(beta_diff_list,axis=0)
          value_sum = np.mean(value_list)
          print(f'cost : {value_sum}')
          if correc_ans is not None:
              prob = prob/episodes
              print(f'prob : {prob}')
          update = self.optimzer.get_updates([QAOA_diff_sum,beta_diff_sum])
          print(f'QAOA update : {update[0]}')
          self.param += np.array(update[0])
          self.b += np.reshape(update[1],-1)
          print(f'QAOA param : {self.param}')




    def rqaoa_execute(self,cal_grad = True):
        ## RQAOA를 확률적 뽑기로 진행해 주는 함수
        Q_init = copy.deepcopy(self.Q)
        Q_action = copy.deepcopy(self.Q)
        self.Q_remove = []
        self.node_assignments = {}  # Keep track of node assignments
        self.edge_expectations = [] # Edge들의 expectation value 저장(Gradient 계산에 필요)
        self.edge_expectations_grad = [] # Edge들의 expectation value gradient 저장(Gradient 계산에 필요)
        self.policys = [] #각 edge가 지정될 확률을 저장(Gradient 계산에 필요)




        QAOA_diff_list = [] ##QAOA layer parameter gradient
        beta_diff_list = [] ##beta parameter gradient
        index = 0
        while Q_init.shape[0] > self.n_c:
            # Select and cut an edge,cal grad
            edge_expectations = self._qaoa_edge_expectations(Q_init,[i for i in range(self.p*index*2,self.p*index*2+2)])
            selected_edge_idx, policy, edge_res = self._select_edge_to_cut(Q_action,Q_init,edge_expectations)
            if cal_grad:
              edge_res_grad = self._qaoa_edge_expectations_gradients(Q_init, [i for i in range(self.p*index*2,self.p*index*2+2)])

            if cal_grad:
              QAOA_diff = self._compute_log_pol_diff(selected_edge_idx,Q_action,edge_res,edge_res_grad ,policy)
              beta_diff = self._compute_grad_beta(selected_edge_idx,Q_action,policy,edge_res)


              QAOA_diff_list.append(QAOA_diff)
              beta_diff_list.append(beta_diff)
            Q_init,Q_action = self._cut_edge(selected_edge_idx,edge_res,Q_action,Q_init)
            index+=1


        self.QAOA_result = copy.deepcopy(self.node_assignments)
        # 그래프의 크기가 n_c보다 작으면 전체 space를 다 계산해서 최종값 도출
        self._brute_force_optimal(Q_init)


        z = self._dict_to_list(self.node_assignments)
        Value = self._state_energy(z,self.Q)

        ## 최근에 얻은 값이 최종 변화율에 영향을 크게 미치도록 함수 변경(큰 그래프에서는 뭘 선택하든 중요하지 않을때가 많음)


        ## 정책에 대한 미분과 panalty 받은 보상을 곱해서 최종 기울기 도출
        if cal_grad:

          beta_diff_res = None
          for beta_diff in beta_diff_list:
            if beta_diff_res is None:
              beta_diff_res = beta_diff
            else:
              beta_diff_res += beta_diff

          QAOA_diff_res = None
          for QAOA_diff in QAOA_diff_list:
            if QAOA_diff_res is None:
              QAOA_diff_res = QAOA_diff
            else:
              QAOA_diff_res += QAOA_diff

          return QAOA_diff_res,beta_diff_res,Value


        else:
          return Value




    def _compute_log_pol_diff(self, idx,Q_action,edge_expectations,edge_expectations_grad,policy):
        ## 특정 edge가 선택 됐을 때 그 정책에 대한 paramter의 log기울기 값 계산
        action_space = self._action_space(Q_action)
        betas = self.b[action_space]
        gather = np.zeros_like(policy)
        for i in range(len(edge_expectations_grad)):
            gather[i] += policy[i] * betas[i]

        diff_log_pol = betas[idx] *np.sign(edge_expectations[idx])* edge_expectations_grad[idx]
        for i in range(len(gather)):
            if gather[i]:
                diff_log_pol-= gather[i] * np.sign(edge_expectations[i]) * (
                    (edge_expectations_grad[i]))

        return np.array(diff_log_pol)

    def _compute_grad_beta(self,idx,Q_action,policy,edge_expectations):

        grad_beta = []
        abs_expectations = abs(np.array(edge_expectations))
        action_space = self._action_space(Q_action)

        betas_idx = action_space

        grad = np.zeros(len(self.b))
        grad[betas_idx[idx]] += abs_expectations[idx]
        for i in range(len(action_space)):
            grad[betas_idx[i]] -= policy[i] * abs_expectations[i]


        return np.array(grad)



    def _cut_edge(self, selected_edge_idx,expectations,Q_action,Q_init):
        ## 선택된 edge에 해당하는 index와 그에 해당하는 expectaion value를 받아 1 혹은 -1 를 edge의 노드에 부여하는 메소드
        ## 또한 Q_action,줄어들어진 Q도 여기서 갱신해준다
        edge_list = [(i, j) for i in range(Q_init.shape[0]) for j in range(Q_init.shape[0]) if Q_init[i, j] != 0 and i!=j]
        edge_to_cut = edge_list[selected_edge_idx]


        expectation = expectations[selected_edge_idx]
        ###수정


        i = edge_to_cut[0]
        j = edge_to_cut[1]

        for key in dict(sorted(self.node_assignments.items(), key=lambda item: item[0])):
          if i >= key:
            i+=1
          if j >= key:
            j+=1

        node_list = list(self.node_assignments.keys())
        node_list += [i,j]
        node_list.sort()


        Q_remove = []
        for index_1 in node_list:
          row = []
          for index_2 in node_list:
            row.append(self.Q[index_1][index_2])
          Q_remove.append(row)
        Q_remove = np.array(Q_remove)


        if expectation < 0:
            self.node_assignments[i] = -1
            self.node_assignments[j] = 1
            sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
            energy_1 = self._state_energy(sorted_dict,Q_remove)


            self.node_assignments[i] = 1
            self.node_assignments[j] = -1
            sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
            energy_2 = self._state_energy(sorted_dict,Q_remove)
            if energy_1 > energy_2:
              self.node_assignments[i] = 1
              self.node_assignments[j] = -1
            else:
              self.node_assignments[i] = -1
              self.node_assignments[j] = 1


        else:
            if Q_init[edge_to_cut[0]][edge_to_cut[0]]+Q_init[edge_to_cut[1]][edge_to_cut[1]]>0:
              # Assign nodes such that they are the same (e.g., 0-0 or 1-1)
              self.node_assignments[i] = -1
              self.node_assignments[j] = -1
              sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
              energy_1 = self._state_energy(sorted_dict,Q_remove)
            else:
              # Assign nodes such that they are the same (e.g., 0-0 or 1-1)
              self.node_assignments[i] = 1
              self.node_assignments[j] = 1
              sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
              energy_2 = self._state_energy(sorted_dict,Q_remove)
              if energy_1 > energy_2:
                self.node_assignments[i] = 1
                self.node_assignments[j] = 1
              else:
                self.node_assignments[i] = -1
                self.node_assignments[j] = -1

        # Remove the nodes connected by the selected edge
        nodes_to_remove = list(edge_to_cut)

        # Create a reduced Q matrix
        remaining_nodes = [i for i in range(Q_init.shape[0]) if i not in nodes_to_remove]
        new_size = len(remaining_nodes)
        new_Q = np.zeros((new_size, new_size))

        for index, node_i in enumerate(remaining_nodes):
            for jedex, node_j in enumerate(remaining_nodes):
                new_Q[index, jedex] = Q_init[node_i, node_j]


        Q_action[i] = 0
        Q_action[:,i] = 0
        Q_action[:,j] = 0
        Q_action[j] = 0



        return new_Q,Q_action

    def _select_edge_to_cut(self,Q_action,Q_init,edge_expectations):
          ## ZZ를 측정하여 beta와 곱한 후 softmax를 진행하여 자를 edge를 확률적으로 고르는 method
          action_space = self._action_space(Q_action)
          interactions = abs(np.array((edge_expectations))) * self.b[action_space]
          probabilities = torch.softmax(torch.tensor(interactions), dim=0).numpy()
          selected_edge_idx = np.random.choice(len(probabilities), p=probabilities)
          return selected_edge_idx, probabilities, edge_expectations




    def _action_space(self,Q_action):
        ## 현재 살아있는 edge들이 기존 그래프에서 몇번째 edge였는지 표시해주는 action space뽑아내주는 함수
        ## Q_action : 죽은 노드들이 있는 행과 열에 대하여 0으로 대체된 살아있는 노드들만 표시되는 매트릭스
        action_space_list = []
        index = 0
        for i in range(Q_action.shape[0]):
            for j in range(Q_action.shape[0]):
                if i!=j and self.Q[i,j] != 0:
                    if Q_action[i,j] != 0:
                        action_space_list.append(index)
                    index+=1
        return action_space_list

    def _qaoa_edge_expectations(self, Q,idx):
        ## 각 edge별 ZZ interaction을 도출해주는 함수

        @qml.qnode(self.qaoa_layer.dev)
        def circuit(param):
            self.qaoa_layer.qaoa_circuit(param)
            return [qml.expval(qml.PauliZ(i) @ qml.PauliZ(j)) for i in range(Q.shape[0]) for j in range(Q.shape[0]) if Q[i, j] != 0 and i!=j]
        return circuit(self.param[idx])

    def _qaoa_edge_expectations_gradients(self, Q,idx):
      ## 각 edge별 ZZ interaction의 기울기를 도출해주는 함수
        res = []
        @qml.qnode(self.qaoa_layer.dev)
        def circuit(params,i,j):
            self.qaoa_layer.qaoa_circuit(params[idx])
            return qml.expval(qml.PauliZ(i) @ qml.PauliZ(j))
        for i in range(Q.shape[0]) :
          for j in range(Q.shape[0]):
             if Q[i, j] != 0 and i != j:
                # Compute gradients for each parameter
                gradients = qml.grad(circuit)(self.param,i,j)
                res.append(gradients)
        return res

    def _brute_force_optimal(self,Q):

        ### 고전적으로 최선의 해를 도출해주는 함수
        if Q.shape[0] > self.n_c:
            print("Graph size too large for brute force.")
            return None

        n = Q.shape[0]
        configs = list(itertools.product([-1, 1], repeat=n))
        best_value = np.inf
        best_config = None



        for config in configs:
            node_assignments = {}
            copy_node = copy.deepcopy(self.node_assignments)


            for index,value in enumerate(config):
              for key in dict(sorted(self.node_assignments.items(), key=lambda item: item[0])):
                if index >= key:
                  index+=1
              node_assignments[index] = value


            copy_node.update(node_assignments)
            z = self._dict_to_list(copy_node)
            value = self._state_energy(z,self.Q)
            if value < best_value:
                best_value = value
                res_node = copy.copy(copy_node)

        self.node_assignments = res_node





    def _state_energy(self,state,Q):
        ## State가 주어질 때 그 에너지를 계산해주는 함수


        # 동일한 크기의 단위 행렬 생성
        identity_matrix = np.eye(Q.shape[0], dtype=bool)

        # 대각선 요소를 제거한 새로운 행렬 생성
        interaction = np.where(identity_matrix, 0, Q)
        diagonal_elements = np.diag(Q)
        value = diagonal_elements@state + state.T@interaction@state
        return value

    def _dict_to_list(self,node_assign):
        ##보조 함수 assign된 node들의 0 과 1을 list 형태로 순서대로 출력해주는 함수
        node_assignments_list = [0]*len(node_assign)
        for key,value in node_assign.items(): # Change: Iterate through key-value pairs using .items()
            node_assignments_list[key]=value
        return np.array(node_assignments_list)




In [23]:
def generate_upper_triangular_qubo(size, low=-10, high=10, integer=True, seed=None):
    """
    Generates an upper-triangular QUBO (Quadratic Unconstrained Binary Optimization) matrix.

    Args:
        size (int): The number of variables (size of the QUBO matrix).
        low (int/float): Minimum value of the random elements.
        high (int/float): Maximum value of the random elements.
        integer (bool): If True, generates integer values; otherwise, generates float values.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: An upper-triangular QUBO matrix of the specified size.
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate random values for the upper triangular part including diagonal
    if integer:
        Q = np.random.randint(low, high, (size, size))
    else:
        Q = np.random.uniform(low, high, (size, size))

    # Keep only the upper triangle values (including diagonal), set lower triangle to zero
    Q = np.triu(Q)

    # Ensure diagonal values are positive (bias terms)
    np.fill_diagonal(Q, np.abs(np.diagonal(Q)))

    return Q

# Example usage
qubo_matrix = generate_upper_triangular_qubo(size=3, low=-5, high=5, integer=True, seed=70)
print("Randomly generated upper-triangular QUBO matrix:\n", qubo_matrix)

import numpy as np

# Original QUBO matrix
qubo_matrix = np.array([
    [0.0658492, -0.024853287,  0.064008761,  0.058160105,  0.058757735],
    [-0.024853287, 0.080128677, -0.025012106, -0.023698929, -0.013849719],
    [0.064008761, -0.025012106,  0.063201845,  0.053540045,  0.055350148],
    [0.058160105, -0.023698929,  0.053540045,  0.066177288,  0.055601435],
    [0.058757735, -0.013849719,  0.055350148,  0.055601435,  0.059764565]
])

diagonal = np.diag(np.diag(qubo_matrix)) + np.eye(qubo_matrix.shape[0])

# Combine results: upper triangle + modified diagonal






# Create an upper triangular matrix by doubling the upper triangle values and keeping diagonal
modified_qubo = np.triu(qubo_matrix, k=1) * 2 + np.diag(np.diag(qubo_matrix))
modified_qubo =  modified_qubo + diagonal

print("Modified QUBO Matrix:\n", modified_qubo)
qubo_matrix = modified_qubo


Randomly generated upper-triangular QUBO matrix:
 [[ 1 -3  3]
 [ 0  1  0]
 [ 0  0  1]]
Modified QUBO Matrix:
 [[ 1.1316984  -0.04970657  0.12801752  0.11632021  0.11751547]
 [ 0.          1.16025735 -0.05002421 -0.04739786 -0.02769944]
 [ 0.          0.          1.12640369  0.10708009  0.1107003 ]
 [ 0.          0.          0.          1.13235458  0.11120287]
 [ 0.          0.          0.          0.          1.11952913]]


In [99]:
import pennylane as qml
from pennylane import numpy as np
from scipy.optimize import minimize

# Define QAOA depth

''' Q = qubo_matrix
print(Q)
depth = 2 '''
Q = generate_upper_triangular_qubo(7,-5,5,integer=False)
depth = 1
# Initialize the QAOA layer
qaoa = QAOA_layer(depth, Q)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)







opt = qml.AdamOptimizer(stepsize=0.1)

# Initial random parameters for QAOA (gammas and betas)
init_params = np.random.uniform(0, np.pi, 2 * depth)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)

# Initial random parameters for QAOA (gammas and betas)
init_params = np.random.uniform(0, np.pi, 2 * depth)

# Optimize the QAOA parameters using classical optimizer (COBYLA)
result = minimize(cost_function, init_params, method="COBYLA", options={'maxiter': 200})

# Extract optimized parameters and minimum cost
optimized_params = result.x
min_cost = result.fun

# Print the results
print("Optimized parameters (gammas and betas):", optimized_params)
print("Minimum cost achieved:", min_cost)


Optimized parameters (gammas and betas): [2.83612748 2.42717376]
Minimum cost achieved: -2.9290448088221472


In [100]:
import pennylane as qml
from pennylane import numpy as np

# Define the QUBO matrix for the problem
def generate_upper_triangular_qubo(size, low=-10, high=10, integer=True, seed=None):
    """
    Generates an upper-triangular QUBO (Quadratic Unconstrained Binary Optimization) matrix.

    Args:
        size (int): The number of variables (size of the QUBO matrix).
        low (int/float): Minimum value of the random elements.
        high (int/float): Maximum value of the random elements.
        integer (bool): If True, generates integer values; otherwise, generates float values.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: An upper-triangular QUBO matrix of the specified size.
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate random values for the upper triangular part including diagonal
    if integer:
        Q = np.random.randint(low, high, (size, size))
    else:
        Q = np.random.uniform(low, high, (size, size))

    # Keep only the upper triangle values (including diagonal), set lower triangle to zero
    Q = np.triu(Q)

    # Ensure diagonal values are positive (bias terms)
    np.fill_diagonal(Q, np.abs(np.diagonal(Q)))

    return Q



# Initialize the QAOA layer
qaoa = QAOA_layer(depth, Q)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)







opt = qml.AdamOptimizer(stepsize=0.1)

# Initial random parameters for QAOA (gammas and betas)
init_params = optimized_params

# Optimization loop
max_iterations = 200
params = init_params
for i in range(max_iterations):
    params = opt.step(cost_function, params)
    if i % 20 == 0:
        cost = cost_function(params)
        print(f"Iteration {i}: Cost = {cost}")

# Final optimized parameters
final_cost = cost_function(params)
print("Optimized parameters (gammas and betas):", params)
print("Final minimum cost achieved:", final_cost)





Iteration 0: Cost = -2.9290448088221472
Iteration 20: Cost = -2.9290448088221472
Iteration 40: Cost = -2.9290448088221472
Iteration 60: Cost = -2.9290448088221472
Iteration 80: Cost = -2.9290448088221472
Iteration 100: Cost = -2.9290448088221472
Iteration 120: Cost = -2.9290448088221472
Iteration 140: Cost = -2.9290448088221472
Iteration 160: Cost = -2.9290448088221472
Iteration 180: Cost = -2.9290448088221472
Optimized parameters (gammas and betas): [2.83612748 2.42717376]
Final minimum cost achieved: -2.9290448088221472


In [101]:
dev = qml.device("default.qubit", shots=1500)
@qml.qnode(dev)
def sample_solution(params):
    """Runs the optimized QAOA circuit and samples the output state."""
    qaoa.qaoa_circuit(params)
    return qml.sample()

# Get multiple samples from the optimized circuit
num_samples = 1000
samples = sample_solution(params)

# Convert measurement results from {-1,1} to {0,1}
binary_solutions = samples*(-2)+1

# Compute the cost for each sampled solution
def evaluate_solution(sample):
    # 동일한 크기의 단위 행렬 생성
    identity_matrix = np.eye(Q.shape[0], dtype=bool)

    # 대각선 요소를 제거한 새로운 행렬 생성
    interaction = np.where(identity_matrix, 0, Q)
    diagonal_elements = np.diag(Q)
    value = diagonal_elements@sample + sample.T@interaction@sample
    return value

# Evaluate the cost for all sampled solutions and find the best
costs = np.array([evaluate_solution(sample) for sample in binary_solutions])
best_solution = binary_solutions[np.argmin(costs)]
best_cost = np.min(costs)

print("Best solution found:", best_solution)
print("Cost of best solution:", best_cost)

Best solution found: [ 1  1 -1 -1 -1 -1 -1]
Cost of best solution: -39.414554192458866


In [102]:
@qml.qnode(dev)
def sample_solution(params):
    """Runs the optimized QAOA circuit and samples the output state."""
    qaoa.qaoa_circuit(params)
    return qml.sample()

In [103]:
# Initial parameters for QAOA
init_params = params
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([25.]*30) ,QAOA_depth=1,gamma = 0.99,learning_rate_init=[0.01,0.05])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )

classical_result : (None, None, tensor(-39.41455419, requires_grad=True)),best : {0: 1, 1: 1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1}


In [104]:
# Evaluate the cost for all sampled solutions and find the best
costs = np.array([evaluate_solution(sample) for sample in binary_solutions])
best_solution = binary_solutions[np.argmin(costs)]
best_cost = np.min(costs)

print("Best solution found:", best_solution)
print("Cost of best solution:", best_cost)
print("Best solution percent : ",np.sum(np.array((costs == float(final_config[2])),dtype=int))/1500)

Best solution found: [ 1  1 -1 -1 -1 -1 -1]
Cost of best solution: -39.414554192458866
Best solution percent :  0.002


In [ ]:

# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = [params]*3
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([25.]*30) ,QAOA_depth=1,gamma = 0.99,learning_rate_init=[0.05,0.00])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-39.41455419, requires_grad=True)),best : {0: 1, 1: 1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1}


 34%|███▍      | 17/50 [00:46<01:30,  2.74s/it]

In [14]:
rl_qaoa.b

tensor([25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25.], requires_grad=True)

In [46]:
rl_qaoa._qaoa_edge_expectations(Q,[0,1])

[tensor(-0.16702916, requires_grad=True),
 tensor(0.10821649, requires_grad=True),
 tensor(0.05888816, requires_grad=True),
 tensor(0.06899251, requires_grad=True),
 tensor(-0.16638491, requires_grad=True),
 tensor(-0.00021, requires_grad=True),
 tensor(-0.12248088, requires_grad=True),
 tensor(-0.36819676, requires_grad=True)]

In [62]:
rl_qaoa._qaoa_edge_expectations(Q,[0,1])

[tensor(-0.02168407, requires_grad=True),
 tensor(-0.07840502, requires_grad=True),
 tensor(0.07189548, requires_grad=True),
 tensor(0.14097381, requires_grad=True),
 tensor(0.01973759, requires_grad=True),
 tensor(-0.01024301, requires_grad=True),
 tensor(-0.25178965, requires_grad=True),
 tensor(-0.25886579, requires_grad=True)]

In [ ]:
rl_qaoa.

In [60]:
rl_qaoa.n_c = 4
final_config = rl_qaoa.rqaoa_execute()


In [61]:
final_config

(tensor([50.40669303,  5.81959104,  0.        ,  0.        ], requires_grad=True),
 tensor([-2.29578388e-05, -2.93378893e-04, -2.68661969e-04,
         -2.84728087e-03, -2.21101320e-05, -8.51516355e-06,
          1.86186705e-01, -1.83475578e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00], requires_grad=True),
 tensor(-23, requires_grad=True))

In [59]:
init_params = [params]*2
init_params = np.reshape(init_params,(-1))
rl_qaoa.param = init_params